In [ ]:
import pandas as pd
import numpy as np

data cleaning

In [ ]:
diabetic = pd.read_csv('../../data/diabetic_data_initial.csv')

In [ ]:
diabetic.shape

In [ ]:
diabetic_df = diabetic.replace('?', np.nan)

In [ ]:
# keep only first encounter for every unique patient
diabetic_df = diabetic_df.sort_values('encounter_id')
diabetic_df.drop_duplicates(subset = ['patient_nbr'], keep = 'first', inplace = True)

In [ ]:
diabetic_df.shape

In [ ]:
# remove expired or hospice-bound patients
diabetic_df = diabetic_df[~diabetic_df['discharge_disposition_id'].isin([11,13,14,19,20,21])]

In [ ]:
# remove unknown-gender patients
diabetic_df.drop(diabetic_df.loc[diabetic_df.gender=='Unknown/Invalid'].index,inplace=True)

In [ ]:
# remove encounter ID, patient NBR, weight, medical specialty, and payer code
diabetic_df.drop(['encounter_id','patient_nbr','weight','medical_specialty','payer_code'],\
                 axis = 1, inplace = True)

In [ ]:
# discretize ID columns
diabetic_df['admission_type_id'] = diabetic_df['admission_type_id'].astype('str')
diabetic_df['admission_source_id'] = diabetic_df['admission_source_id'].astype('str')
diabetic_df['discharge_disposition_id'] = diabetic_df['discharge_disposition_id'].astype('str')

In [ ]:
diabetic_df.shape

feature engineering

In [ ]:
# binarize target variable: readmitted < 30 days or other
diabetic_df['readmit_30d'] = (diabetic_df['readmitted'] == '<30')

In [ ]:
# convert age to a continuous variable
diabetic_df['age_num'] = diabetic_df['age'].replace(
    ['[0-10)','[10-20)','[20-30)','[30-40)','[40-50)','[50-60)','[60-70)','[70-80)','[80-90)','[90-100)'],
    [0,10,20,30,40,50,60,70,80,90])

In [ ]:
# keep main disease class only
diabetic_df['diag_1'] = diabetic_df['diag_1'].str.split('.', expand = True).drop(1, axis = 1)
diabetic_df['diag_2'] = diabetic_df['diag_2'].str.split('.', expand = True).drop(1, axis = 1)
diabetic_df['diag_3'] = diabetic_df['diag_3'].str.split('.', expand = True).drop(1, axis = 1)

In [ ]:
# create a binary variable for each diagnosis in diag_1,2, or 3 that has sample size >500
count_1 = diabetic_df['diag_1'].value_counts()
index_1 = count_1[count_1>=500].index.tolist()

count_2 = diabetic_df['diag_2'].value_counts()
index_2 = count_2[count_2>=500].index.tolist()

count_3 = diabetic_df['diag_3'].value_counts()
index_3 = count_3[count_3>=500].index.tolist()

diagnoses = set(index_1 + index_2 + index_3)

for d in diagnoses:
    diabetic_df[d+'_diag'] = ((diabetic_df['diag_1']==d)|
                              (diabetic_df['diag_2']==d)|
                              (diabetic_df['diag_3']==d))

In [ ]:
# binarize medication features with minority classes > 500 samples: used/not used
for col in ['metformin','repaglinide','glimepiride','glipizide', 'glyburide','pioglitazone',
            'rosiglitazone','insulin']:
    diabetic_df[col+'_used'] = np.where(diabetic_df[col]=='No', False, True)

In [ ]:
# remove features used to engineer new features
diabetic_final = diabetic_df.drop(['age',
                                   'diag_1','diag_2','diag_3',
                                   'metformin','repaglinide','nateglinide','chlorpropamide','glimepiride',
                                   'acetohexamide','glipizide','glyburide','tolbutamide','pioglitazone',
                                   'rosiglitazone','acarbose','miglitol','troglitazone','tolazamide','examide',
                                   'citoglipton','insulin','glyburide-metformin','glipizide-metformin',
                                   'glimepiride-pioglitazone','metformin-rosiglitazone','metformin-pioglitazone',
                                   'readmitted'],
                                 axis = 1)

In [ ]:
diabetic_final.columns

In [ ]:
# dummify features
diabetic_final = pd.get_dummies(diabetic_final, drop_first = True)
diabetic_final.shape

export

In [ ]:
diabetic_final.to_csv('diabetic_model_set.csv')